In [ ]:
# ============================================================
# 1️⃣ INSTALLS
# ============================================================
!pip install kagglehub pandas numpy scikit-learn tensorflow pyswarms prettytable


In [ ]:
# ============================================
# CELL 1 — IMPORTS
# ============================================
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
import pyswarms as ps
from pyswarms.single.global_best import GlobalBestPSO
from prettytable import PrettyTable
import kagglehub

tf.random.set_seed(42)
np.random.seed(42)


In [ ]:
# ============================================
# CELL 2 — DOWNLOAD + LOAD + SAMPLE DATASET
# ============================================

# Download dataset automatically
path = kagglehub.dataset_download("chethuhn/network-intrusion-dataset")
print("Dataset downloaded to:", path)

# Load all CSV files
csv_files = [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".csv")]
print("Found CSV files:", csv_files)

df = pd.concat([pd.read_csv(f, low_memory=False) for f in csv_files],
               ignore_index=True)

print("Full dataset shape:", df.shape)

# -------------------------------------------------------
# 📌 RAM SAVING FIX – SAMPLE BEFORE ANY PROCESSING
# -------------------------------------------------------
SAMPLE_FRACTION = 0.05  # 5% of the data to avoid RAM crash (change to 0.01 if needed)

df = df.sample(frac=SAMPLE_FRACTION, random_state=42).reset_index(drop=True)
print("After sampling:", df.shape)


Using Colab cache for faster access to the 'network-intrusion-dataset' dataset.
Dataset downloaded to: /kaggle/input/network-intrusion-dataset
Found CSV files: ['/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Tuesday-WorkingHours.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Monday-WorkingHours.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Friday-WorkingHours-Morning.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv', '/kaggle/input/network-intrusion-dataset/Wednesday-workingHours.pcap_ISCX.csv']
Full dataset shape: (2830743, 79)
After sampling: (141537, 79)


In [ ]:
# ============================================
# CELL 3 — CLEANING + PREPROCESSING + SPLIT
# ============================================

# Standardize column names
df.columns = df.columns.str.strip().str.lower()

# Convert label to numeric
df["label"] = df["label"].astype("category").cat.codes

# Replace infinite values
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop any rows with NaN
df.dropna(inplace=True)

# Separate X and y
X = df.drop(columns=["label"])
y = df["label"]

# ============================================================
# FIX: Remove classes with fewer than 2 samples
# ============================================================
class_counts = y.value_counts()
valid_classes = class_counts[class_counts >= 2].index
df = df[df["label"].isin(valid_classes)]

# Re-separate X and y
X = df.drop(columns=["label"])
y = df["label"]

print("Classes remaining:", y.nunique())
print("Samples per class:\n", y.value_counts())

# Scale features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train / Validation / Test Split
X_train, X_temp, y_train, y_temp = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

# Convert labels to NumPy
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

print("X_train:", X_train.shape)
print("X_test:", X_test.shape)


Classes remaining: 13
Samples per class:
 label
0     113651
4      11499
10      7877
2       6391
3        516
7        424
6        293
11       289
5        260
1         83
12        70
13        41
8          2
Name: count, dtype: int64
X_train: (113116, 78)
X_test: (14140, 78)


In [ ]:
# ============================================
# CELL 4 — EXTREME LEARNING MACHINE CLASS
# ============================================

class ELM:
    def __init__(self, input_dim, hidden_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.W = np.random.randn(self.input_dim, self.hidden_dim)
        self.b = np.random.randn(self.hidden_dim)

    def forward(self, X):
        return np.tanh(X @ self.W + self.b)

    def fit(self, X, y):
        H = self.forward(X)
        self.beta = np.linalg.pinv(H) @ y

    def predict(self, X):
        H = self.forward(X)
        return (H @ self.beta > 0.5).astype(int)


In [ ]:
# ============================================
# CELL 5 — PSO OPTIMIZATION OF ELM
# ============================================

X_train_fs = X_train
y_train_fs = y_train

input_dim = X_train_fs.shape[1]
hidden_dim = 120  # same as research paper

param_dim = (input_dim * hidden_dim) + hidden_dim

def fitness_function(particles):
    losses = []
    for particle in particles:
        # Extract weights + biases
        W_flat = particle[:input_dim * hidden_dim]
        b_flat = particle[input_dim * hidden_dim:]

        W = W_flat.reshape(input_dim, hidden_dim)
        b = b_flat

        H = np.tanh(X_train_fs @ W + b)
        beta = np.linalg.pinv(H) @ y_train_fs
        pred = (H @ beta > 0.5).astype(int)

        acc = (pred == y_train_fs).mean()
        losses.append(1 - acc)

    return np.array(losses)

# PSO optimizer settings
optimizer = GlobalBestPSO(
    n_particles=20,
    dimensions=param_dim,
    options={"c1": 2.0, "c2": 2.0, "w": 0.9}
)

best_cost, best_pos = optimizer.optimize(fitness_function, iters=10)

print("PSO optimization complete.")


2025-11-25 13:55:40,646 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 2.0, 'c2': 2.0, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|10/10, best_cost=0.35
2025-11-25 14:06:09,687 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.3501096219809753, best pos: [ 2.35631764 -0.74537805  4.5428027  ...  0.40179629  3.78631142
 -0.29026125]


PSO optimization complete.


In [ ]:
# ============================================
# CELL 6 — FINAL ELM TRAINING
# ============================================

W_flat = best_pos[:input_dim * hidden_dim]
b_flat = best_pos[input_dim * hidden_dim:]

W = W_flat.reshape(input_dim, hidden_dim)
b = b_flat

elm = ELM(input_dim, hidden_dim)
elm.W = W
elm.b = b

elm.fit(X_train_fs, y_train_fs)

pred_test = elm.predict(X_test)


In [ ]:
# ============================================
# CELL 7 — EVALUATION
# ============================================

print("\nCONFUSION MATRIX")
print(confusion_matrix(y_test, pred_test))

print("\nCLASSIFICATION REPORT")
print(classification_report(y_test, pred_test))



CONFUSION MATRIX
[[9243 2123    0    0    0    0    0    0    0    0    0    0]
 [   4    4    0    0    0    0    0    0    0    0    0    0]
 [   1  638    0    0    0    0    0    0    0    0    0    0]
 [   3   48    0    0    0    0    0    0    0    0    0    0]
 [   1 1149    0    0    0    0    0    0    0    0    0    0]
 [   7   19    0    0    0    0    0    0    0    0    0    0]
 [   3   27    0    0    0    0    0    0    0    0    0    0]
 [   0   43    0    0    0    0    0    0    0    0    0    0]
 [   0  787    0    0    0    0    0    0    0    0    0    0]
 [  11   18    0    0    0    0    0    0    0    0    0    0]
 [   0    7    0    0    0    0    0    0    0    0    0    0]
 [   0    4    0    0    0    0    0    0    0    0    0    0]]

CLASSIFICATION REPORT
              precision    recall  f1-score   support

           0       1.00      0.81      0.90     11366
           1       0.00      0.50      0.00         8
           2       0.00      0.00      

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
